In [1]:
!pip install --quiet psycopg2

In [12]:
import pandas as pd
import re
import requests
import psycopg2 as pg2
from psycopg2.extras import RealDictCursor





In [6]:
def cleaner(message):
    message = re.sub('\.+', ' ', message)
    message = re.sub('[^a-z0-9 ]',' ', message.lower())
    message = re.sub('\d+',' NUMBER ',message)
    message = re.sub('\s+',' ',message)
    return message 

In [ ]:
test = 'i like ice-cream  and cookies.'

re.sub('\s+', ' ', test)

In [ ]:
!ls ../docker/postgres/data

In [4]:
ml_pages_df[ ml_pages_df.pageid == 27116291]

NameError: name 'ml_pages_df' is not defined

In [4]:
ml_pages_df = pd.read_pickle('../docker/postgres/data/ml.p')

In [5]:
ml_cleaned_df = ml_pages_df.copy()

ml_cleaned_df.pageid = ml_cleaned_df.pageid.apply( lambda x: int(x))
ml_cleaned_df.text = ml_cleaned_df.text.apply( cleaner)
ml_cleaned_df.title = ml_cleaned_df.title.apply( cleaner)


ml_cleaned_df.head()

,pageid,subcategory,title,category,text
0,43385931,machine learning,data exploration,machine learning,data exploration is an approach similar to ini...
1,49082762,machine learning,list of datasets for machine learning research,machine learning,these datasets are used for machine learning r...
2,233488,machine learning,machine learning,machine learning,machine learning is the subfield of computer s...
3,53587467,machine learning,outline of machine learning,machine learning,the following outline is provided as an overvi...
4,3771060,machine learning,accuracy paradox,machine learning,the accuracy paradox for predictive analytics ...


In [137]:
ml_cleaned_df.to_pickle('../docker/postgres/data/ml_cleaned.p')

In [138]:
!ls ../docker/postgres/data/

bs_cleaned.p  ml_cleaned.p  ml.p


In [136]:
ml_cleaned_df.shape

(1304, 5)

In [6]:
bs_cleaned_df = pd.read_pickle('../docker/postgres/data/bs_cleaned.p')

In [7]:
bs_cleaned_df.head()

,pageid,subcategory,title,category,text
0,1037763,business software,business software,business software,business software or a business application is...
1,41270069,business software,accusystems,business software,accusystems llc is an american company headqua...
2,5211212,business software,active policy management,business software,active policy management is business oriented ...
3,28502793,business software,alexandria library software,business software,alexandria is browser based cross platform lib...
4,44133735,business software,alteryx,business software,alteryx is an american computer software compa...


In [ ]:
ml_pages_df.loc[:,'text'] = ml_pages_df.text.str.replace('\n',' ')
ml_pages_df.loc[:,'text'] = ml_pages_df.text.str.replace("\'", '')



In [ ]:
ml_pages_df.iloc[2,:].text

In [8]:
def connect_to_db():
    con = pg2.connect(host='postgres',
                      dbname='wikipedia',
                      user='postgres')
    cur = con.cursor(cursor_factory=RealDictCursor)
    return con, cur

In [9]:
def query_to_dictionary(query, fetch_res=True):
    con, cur = connect_to_db()
    cur.execute(query)
    if fetch_res:
        results = cur.fetchall()
    else:
        results = None
    con.close()
    return results

In [20]:
def query_to_dataframe(query):
    return pd.DataFrame(query_to_dictionary(query))

In [ ]:
!pip install psycopg2




con, cur = connect_to_db

con = pg2.connect(host='postgres',  #127.0.0.1
                      dbname='wikipedia',
                      user='postgres')
cursor = con.cursor(cursor_factory=RealDictCursor)

In [13]:
def update_page_table(pageid, title, text):
    query = """BEGIN;INSERT INTO pages (pageid, title, article) VALUES ({}, '{}', '{}');COMMIT;""".format(pageid, title, text )
    
    return query

In [17]:
results = ml_cleaned_df.apply( lambda x: query_to_dictionary(update_page_table( x.pageid, x.title, x.text), fetch_res=False), axis = 1)

In [118]:
results = bs_cleaned_df.apply( lambda x: query_to_dictionary(update_page_table( x.pageid, x.title, x.text), fetch_res=False), axis = 1)

In [28]:
def update_category_table(category):
    query = """BEGIN;INSERT INTO categories (category) VALUES ('{}');COMMIT;""".format(category )
    #ALTER TABLE category category_id SERIAL PRIMARY KEY;
    
    return query

In [27]:
categories_df = ml_cleaned_df[['category']].drop_duplicates().copy()
categories_df

,category
0,machine learning


In [119]:
categories_df = bs_cleaned_df[['category']].drop_duplicates().copy()
categories_df

,category
0,business software


In [29]:
results = categories_df.apply( lambda x: query_to_dictionary(update_category_table( x.category), fetch_res=False), axis = 1)

In [120]:
results = categories_df.apply( lambda x: query_to_dictionary(update_category_table( x.category), fetch_res=False), axis = 1)

In [122]:
catTable_df = query_to_dataframe("SELECT * FROM categories;")
catTable_df

,category,category_id
0,machine learning,1
1,business software,2


In [39]:
def update_sub_category_table(subcategory, category):
    query = """BEGIN;INSERT INTO subcategories (subcategory, category_id) VALUES ('{}',(SELECT category_id FROM categories WHERE category = '{}'));COMMIT;""".format(subcategory, category )
    #ALTER TABLE category category_id SERIAL PRIMARY KEY;
    
    return query

In [31]:
subcats_df = ml_cleaned_df[['subcategory']].drop_duplicates().copy()
subcats_df

,subcategory
0,machine learning
195,Applied machine learning
229,Artificial neural networks
357,Deep learning
376,Neural network software
398,Bayesian networks
410,Classification algorithms
491,Decision trees
508,Ensemble learning
522,Cluster analysis


In [123]:
subcats_df = bs_cleaned_df[['subcategory']].drop_duplicates().copy()
subcats_df

,subcategory
0,business software
296,Administrative software
318,School-administration software
340,Time-tracking software
365,Business simulation games
553,Amusement park simulation games
571,M.U.L.E.
574,Roller coaster games and simulations
597,Sports management video games
633,Association football management video games


In [40]:
results = subcats_df.apply( lambda x: query_to_dictionary(update_sub_category_table( x.subcategory, 'machine learning'), fetch_res=False), axis = 1)

In [124]:
results = subcats_df.apply( lambda x: query_to_dictionary(update_sub_category_table( x.subcategory, 'business software'), fetch_res=False), axis = 1)

In [42]:
query_to_dataframe('SELECT category, subcategory, subcategory_id FROM subcategories sc JOIN categories c ON sc.category_id = c.category_id;')

,category,subcategory
0,machine learning,machine learning
1,machine learning,Applied machine learning
2,machine learning,Artificial neural networks
3,machine learning,Deep learning
4,machine learning,Neural network software
5,machine learning,Bayesian networks
6,machine learning,Classification algorithms
7,machine learning,Decision trees
8,machine learning,Ensemble learning
9,machine learning,Cluster analysis


In [44]:
def update_page_category_table( pageid, subcategory):
    query = """BEGIN;INSERT INTO page_category (pageid, subcategory_id) VALUES ({}, (SELECT subcategory_id FROM subcategories WHERE subcategory = '{}'));COMMIT;""".format( pageid, subcategory)
    return query

In [45]:
results = ml_cleaned_df.apply( lambda x: query_to_dictionary( update_page_category_table( x.pageid, x.subcategory), fetch_res = False), axis =1) 

In [125]:
results = bs_cleaned_df.apply( lambda x: query_to_dictionary( update_page_category_table( x.pageid, x.subcategory), fetch_res = False), axis =1) 

In [ ]:
query = """SELECT category, subcategory, pageid, title 
            FROM pages p 
            JOIN page_category pc 
            ON p.pageid = pc.pageid"""

query_to_dataframe('SELECT ')

In [ ]:
query1 = '(SELECT category, subcategory, subcategory_id FROM subcategories sc JOIN categories c ON sc.category_id = c.category_id) as stuff'

In [ ]:
query_to_dataframe('SELECT pageid')

In [ ]:
cat_subcats_df.apply( lambda x: cursor.execute( update_category_table( x.category, x.subcategory)), axis = 1)

In [23]:
cat_subcats_df = ml_cleaned_df[['category','subcategory']].drop_duplicates().copy()
cat_subcats_df

,category,subcategory
0,machine learning,machine learning
195,machine learning,Applied machine learning
229,machine learning,Artificial neural networks
357,machine learning,Deep learning
376,machine learning,Neural network software
398,machine learning,Bayesian networks
410,machine learning,Classification algorithms
491,machine learning,Decision trees
508,machine learning,Ensemble learning
522,machine learning,Cluster analysis


In [ ]:
SELECT category, subcategory, pageid, title 

page_category pc 
    JOIN pages p ON pc.pageid = p.pageid
    JOIN (SELECT category, subcategory)  

In [ ]:
SELECT stuff.category, stuff.subcategory, p.pageid, p.title
    SELECT stuff.category, stuff.subcategory, pc.pageid
    FROM (SELECT category, subcategory, subcategory_id 
            FROM subcategories sc 
            JOIN categories c 
            ON sc.category_id = c.category_id) as stuff
    JOIN page_category pc
    ON stuff.subcategory_id = pc.subcategory_id
JOIN pages p
ON pc.pageid = p.pageid

In [133]:
innerquery = """SELECT stuff.category, stuff.subcategory, pc.pageid
    FROM (SELECT category, subcategory, subcategory_id 
            FROM subcategories sc 
            JOIN categories c 
            ON sc.category_id = c.category_id) as stuff
    JOIN page_category pc
    ON stuff.subcategory_id = pc.subcategory_id"""

innerquery = re.sub( "\s+"," ", innerquery)
innerquery

'SELECT stuff.category, stuff.subcategory, pc.pageid FROM (SELECT category, subcategory, subcategory_id FROM subcategories sc JOIN categories c ON sc.category_id = c.category_id) as stuff JOIN page_category pc ON stuff.subcategory_id = pc.subcategory_id'

In [128]:
outerquery = """SELECT DISTINCT subcategory, p.pageid, p.title 
                FROM ({}) as crap
                JOIN pages p
                ON crap.pageid = p.pageid
                WHERE category = 'business software'
                ORDER BY subcategory ASC, title ASC;""".format( innerquery)

outerquery = re.sub("\s+", " ", outerquery)
outerquery

##  WHERE p.title = 'draft decision stream'
##WHERE subcategory = 'Classification algorithms'

"SELECT DISTINCT subcategory, p.pageid, p.title FROM (SELECT stuff.category, stuff.subcategory, pc.pageid FROM (SELECT category, subcategory, subcategory_id FROM subcategories sc JOIN categories c ON sc.category_id = c.category_id) as stuff JOIN page_category pc ON stuff.subcategory_id = pc.subcategory_id) as crap JOIN pages p ON crap.pageid = p.pageid WHERE category = 'business software' ORDER BY subcategory ASC, title ASC;"

In [134]:
countquery = """SELECT category, COUNT(p.title)
                FROM ({}) as crap
                JOIN pages p
                ON crap.pageid = p.pageid
                GROUP BY category;""".format( innerquery)

countquery = re.sub("\s+", " ", countquery)

In [117]:
print(query_to_dataframe( "SELECT DISTINCT title FROM pages WHERE title LIKE 'd%' ORDER BY title ASC;"))

                                               title
0                                             dadisp
1                                         darkforest
2                                 darpa lagr program
3                                 dartmouth workshop
4                                        darwintunes
5                                      datadetective
6                                   data exploration
7                                            dataiku
8                             data mining extensions
9                                data pre processing
10                            data stream clustering
11                              davies bouldin index
12                                            dbscan
13                                 decision boundary
14                                     decision list
15                                    decision stump
16                                     decision tree
17                            decision tree le

In [135]:
query_to_dataframe( countquery)

,category,count
0,business software,12276
1,machine learning,1867


In [129]:
query_to_dataframe(outerquery)

,pageid,subcategory,title
0,2920499,Abstract management software,abstract management
1,16329575,Abstract management software,easychair
2,36306221,Abstract management software,exordo
3,24702350,Abstract management software,openconf
4,45039676,Abstract management software,open conference systems
5,1948086,Account aggregation providers,account aggregation
6,29281338,Account aggregation providers,check mobile app
7,24764216,Account aggregation providers,financeworks
8,13783191,Account aggregation providers,mint com
9,5764847,Account aggregation providers,yodlee


In [81]:
query_to_dataframe(innerquery+";")

,category,pageid,subcategory
0,machine learning,43385931,machine learning
1,machine learning,49082762,machine learning
2,machine learning,233488,machine learning
3,machine learning,53587467,machine learning
4,machine learning,3771060,machine learning
5,machine learning,43808044,machine learning
6,machine learning,28801798,machine learning
7,machine learning,45049676,machine learning
8,machine learning,52642349,machine learning
9,machine learning,30511763,machine learning


In [ ]:
ml_cleaned_df.subcategory.unique()

In [ ]:
import numpy as np

In [ ]:
ml_cleaned_df.head()

In [ ]:
test = ml_cleaned_df.iloc[0,]
test

In [ ]:
cursor.execute( update_category_table(test.category))

In [ ]:
delete_query = """BEGIN;DELETE FROM category;COMMIT;"""

In [ ]:
query_to_dictionary(delete_query, fetch_res = False )

In [18]:
con = pg2.connect(host='postgres',  #127.0.0.1
                      dbname='wikipedia',
                      user='postgres')
cursor = con.cursor(cursor_factory=RealDictCursor)

In [16]:
cursor.execute('SELECT * FROM categories;')

cursor.fetchall()

[]

In [26]:
query_to_dataframe('SELECT pageid, title FROM pages LIMIT 5;')


,pageid,title
0,43385931,data exploration
1,49082762,list of datasets for machine learning research
2,233488,machine learning
3,53587467,outline of machine learning
4,3771060,accuracy paradox


In [ ]:
ml_pages_df.loc[167, :].apply(lambda x: "'" in x["text"])

In [ ]:
cat_subcats_df.apply( lambda x: cursor.execute( update_category_table( x.category, x.subcategory)), axis = 1)

In [ ]:
ml_cleaned_df[ml_cleaned_df.pageid == 405562]['title'][167]

In [ ]:
!pip install sqlalchemy

In [ ]:
from sqlalchemy import create_engine

engine = create_engine( "postgresql://postgres@postgres/postgres")

In [ ]:
df.to_sql("<table name>", con=engine, it_exists="replace", index = False)

In [ ]:
pd.read_sql( sql, con=engine)

In [ ]:

## Also return level of subcategory

def get_subcategories( category, depth =4, subcategories = [], catlevel = []):  ## Restrixt depth to level 4
    
    
    category_df = request_elements( category, 'subcat')
    
    #cat_labels = category + "/"
    
    for subcat in category_df.title.str.replace('Category:', '').tolist():
        
        if subcat not in subcategories:
            #print( subcat)
            subcategories.append( subcat)
            catlevel.append( depth)
            try:
                if depth < 0:
                    #print("BREAK")
                    break
                else:
                    subcategory_depth = depth - 1
                    #subcategories = get_subcategories( subcat,depth - 1, subcategories)
                    subcategories, catlevel = get_subcategories( subcat,subcategory_depth, subcategories, catlevel)
            except:
                continue
    return subcategories, catlevel
            

In [ ]:
subcategories, catlevel = get_subcategories( 'machine learning', 4, subcategories = [], catlevel = [])

In [13]:
def category_format( category):
    category_query = re.sub( '\s', '+', category)
    return category_query
    

In [14]:
def format_query(category, ptype ):
    '''Category should be provided as a string,  ptype may be page, subcat, or file'''
    categoryF = category_format(category)
    
    ptype_dict = {'page':'0', 'subcat':'14','file':'6'}
    nstype = ptype_dict[ ptype]
    base_url = 'https://en.wikipedia.org/w/api.php'
    action_tag = "?action=query&list=categorymembers&cmlimit=max" ## fetch all category members (pages, subcategories)
    category_tag = '&cmtitle=Category:{}&cmtype={}&cmnamespace={}&format=json'.format( categoryF, ptype, nstype) ## append category to cat_tag
    query = base_url + action_tag + category_tag# + parameters_tag ## concatenate base_url with request tags
    return query

In [15]:
def request_elements( category, ptype, tag = False):
    query = format_query( category, ptype)
    
    r = requests.get( query)  ## request HTTP results
    response = r.json()

    elements_df = pd.DataFrame( response['query']['categorymembers'])
    if tag:
        elements_df['subcategory'] = tag 
    return elements_df
    

In [76]:
## THIS WORKS

def get_subcategories1( category, depth =4, first_run = True): #, visited = []):  ## Restrixt depth to level 4
    
    category_df = request_elements( category, 'subcat')
    
    #subcats = [cat for cat in category_df.title.str.replace('Category:', '').tolist() if cat not in visited]
    
    #print( subcats)
    
    #visited += subcats
    
    catdict = {}
    
    if category_df.empty:
        return catdict
    else:
        subcats = [cat for cat in category_df.title.str.replace('Category:', '').tolist()] # if cat not in visited]
    #catdict[category] =  subcats
    
    #cat_labels = category + "/"
    
    for subcat in subcats:
        try:
            if depth < 0:
                break
            else:
                catdict[subcat] = get_subcategories1( subcat, depth -1, first_run = False) #, visited = visited)
                #subcatdict = catdict[category][subcat]
            #try:
            #    if depth < 0:
            #        #print("BREAK")
            #        break
            #    else:
            #        subcategories, catdict = get_subcategories( subcat,depth - 1, subcategories, subcatdict)
        except:
            continue
    if first_run:
        return {category: catdict}
    else:
        return catdict
    #return catdict
            

In [77]:
cats = get_subcategories1( 'machine learning')

In [78]:
cats

{'machine learning': {'Applied machine learning': {},
  'Artificial intelligence conferences': {},
  'Artificial neural networks': {'Deep learning': {},
   'Neural network software': {}},
  'Bayesian networks': {},
  'Classification algorithms': {'Artificial neural networks': {'Deep learning': {},
    'Neural network software': {}},
   'Decision trees': {},
   'Ensemble learning': {}},
  'Cluster analysis': {'Cluster analysis algorithms': {},
   'Clustering criteria': {}},
  'Computational learning theory': {},
  'Data mining and machine learning software': {'Social network analysis software': {}},
  'Datasets in machine learning': {'Datasets in computer vision': {}},
  'Dimension reduction': {'Factor analysis': {}},
  'Ensemble learning': {},
  'Evolutionary algorithms': {'Gene expression programming': {},
   'Genetic algorithms': {'Artificial immune systems': {},
    'Gene expression programming': {}},
   'Genetic programming': {},
   'Nature-inspired metaheuristics': {}},
  'Genetic

In [38]:
## original
def get_cat_structure( cat_name, first_run = True): #, visited = []):
    
    category_df = request_elements( cat_name, 'subcat')

    #visited += subcats 
    
    output = {}
    
    if category_df.empty:
        return output
    else:
        subcats = [cat for cat in category_df.title.str.replace('Category:', '').tolist()] # if cat not in visited]
        

    
    for cat in subcats:
        try:
            output[ cat] = get_cat_structure( cat, first_run = False)#, visited = visited )
        except: 
            print( cat)
            continue
    if first_run:
        #print({cat_name: output})
        return {cat_name: output}
    
    
    else:
        return output

In [ ]:
## New

def get_cat_structure( cat_name, first_run = True): #, visited = []):
    
    category_df = request_elements( cat_name, 'subcat')
    
    pages_df = request_elements( category, 'page', category)

    #visited += subcats 
    
    output = {}
    
    if category_df.empty:
        return output
    else:
        subcats = [cat for cat in category_df.title.str.replace('Category:', '').tolist()] # if cat not in visited]
        

    
    for cat in subcats:
        try:
            output[ cat] = get_cat_structure( cat, first_run = False)#, visited = visited )
        except: 
            print( cat)
            continue
    if first_run:
        #print({cat_name: output})
        return {cat_name: output}
    
    
    else:
        return output

In [22]:
cat_name = 'machine learning'
category_df = request_elements( cat_name, 'subcat')



subcats = [cat for cat in category_df.title.str.replace('Category:', '').tolist() if cat not in visited]
#category_df.head(), 
subcats[0:5]

(   ns    pageid                                title
 0  14  33547387    Category:Applied machine learning
 1  14  42936114  Category:Artificial neural networks
 2  14   1718975           Category:Bayesian networks
 3  14   1991254   Category:Classification algorithms
 4  14  22532673            Category:Cluster analysis,
 ['Applied machine learning',
  'Artificial neural networks',
  'Bayesian networks',
  'Classification algorithms',
  'Cluster analysis'])

In [37]:
test_df = request_elements('Applied machine learning', 'subcat')
if test_df.empty:
    print( 'true empty')

true empty


In [67]:
test = get_cat_structure( 'machine learning')

In [45]:
print('Parent: {}'.format( test.keys()) )
print( 'Child1: {}'.format( len( test['machine learning'].keys())) )
print( 'Child2: {}'.format( test['machine learning']['Artificial neural networks'].keys())) 


Parent: dict_keys(['machine learning'])
Child1: 30
Child2: dict_keys(['Deep learning', 'Neural network software'])


SyntaxError: invalid syntax (<ipython-input-71-a58a88215da2>, line 1)

In [72]:
A = {}
if A:
    print('empty')

In [81]:
test_list = []

for cat, subcats in test.items():

    #print( cat)
    for subcat, sub_subcats in subcats.items():
        #print( '\tL1: ' + subcat)
        if sub_subcats:
            for sub_subcat, sub_sub_subcats in sub_subcats.items():
                #print( '\t\tL2: ' + sub_subcat)
                if sub_sub_subcats:
                
                
                    for sub_sub_subcat, sub_sub_sub_subs in sub_sub_subcats.items():
                        
                        test_list.append( {'parent': cat, 'child1':subcat , 'child2':sub_subcat, 'child3':sub_sub_subcat})
                        #print( '\t\t\tL3: ' + sub_sub_subcat)
                else:
                    test_list.append( {'parent': cat, 'child1':subcat , 'child2':sub_subcat, 'child3':"-"})

        else:
            test_list.append( {'parent': cat, 'child1':subcat , 'child2':"-", 'child3':"-"})
                
test_df = pd.DataFrame(test_list)                
test_df[['parent', 'child1', 'child2', 'child3']].head(15)

,parent,child1,child2,child3
0,machine learning,Applied machine learning,-,-
1,machine learning,Artificial neural networks,Deep learning,-
2,machine learning,Artificial neural networks,Neural network software,-
3,machine learning,Bayesian networks,-,-
4,machine learning,Classification algorithms,Artificial neural networks,Deep learning
5,machine learning,Classification algorithms,Artificial neural networks,Neural network software
6,machine learning,Classification algorithms,Decision trees,-
7,machine learning,Classification algorithms,Ensemble learning,-
8,machine learning,Cluster analysis,Cluster analysis algorithms,-
9,machine learning,Cluster analysis,Clustering criteria,-


In [ ]:
test_list1 = []

for cat, subcats in test.items():

    #print( cat)
    for subcat, sub_subcats in subcats.items():
        #print( '\tL1: ' + subcat)
        if sub_subcats:
            for sub_subcat, sub_sub_subcats in sub_subcats.items():
                #print( '\t\tL2: ' + sub_subcat)
                if sub_sub_subcats:
                
                
                    for sub_sub_subcat, sub_sub_sub_subs in sub_sub_subcats.items():
                        
                        test_list1.append( {'parent': cat, 'child1':subcat , 'child2':sub_subcat, 'child3':sub_sub_subcat})
                        #print( '\t\t\tL3: ' + sub_sub_subcat)
                else:
                    test_list1.append( {'category': cat,  'subcategory'+ "/" + subcat + "/" + sub_subcat, 'child3':"-"})

        else:
            test_list1.append( {'parent': cat, 'child1':subcat , 'child2':"-", 'child3':"-"})
                
test_df1 = pd.DataFrame(test_list1)                
test_df1

In [63]:
test_df = pd.DataFrame( test)
test_df.head()

,machine learning
Applied machine learning,{}
Artificial intelligence conferences,{}
Artificial neural networks,"{'Deep learning': {}, 'Neural network software..."
Bayesian networks,{}
Classification algorithms,{'Artificial neural networks': {'Deep learning...


In [ ]:
for k, v in test.items()

In [65]:
dict_of_df = {k: pd.DataFrame(v) for k,v in test.items()}
df = pd.concat(dict_of_df, axis=1)
df

machine learning  \
                                 Applied machine learning   
Artificial neural networks                            NaN   
Cluster analysis algorithms                           NaN   
Clustering criteria                                   NaN   
Datasets in computer vision                           NaN   
Decision trees                                        NaN   
Deep learning                                         NaN   
Ensemble learning                                     NaN   
Factor analysis                                       NaN   
Gene expression programming                           NaN   
Genetic algorithms                                    NaN   
Genetic programming                                   NaN   
Graphical models                                      NaN   
Hidden Markov models                                  NaN   
Language modeling                                     NaN   
Markov networks                                       NaN   
Nature-inspired metaheuristics                        NaN   
Neural network software                               NaN   
Social network analysis software                      NaN   
Structural equation models                            NaN   
Support vector machines                               NaN   

                                                                      \
                                 Artificial intelligence conferences   
Artificial neural networks                                       NaN   
Cluster analysis algorithms                                      NaN   
Clustering criteria                                              NaN   
Datasets in computer vision                                      NaN   
Decision trees                                                   NaN   
Deep learning                                                    NaN   
Ensemble learning                                                NaN   
Factor analysis                                                  NaN   
Gene expression programming                                      NaN   
Genetic algorithms                                               NaN   
Genetic programming                                              NaN   
Graphical models                                                 NaN   
Hidden Markov models                                             NaN   
Language modeling                                                NaN   
Markov networks                                                  NaN   
Nature-inspired metaheuristics                                   NaN   
Neural network software                                          NaN   
Social network analysis software                                 NaN   
Structural equation models                                       NaN   
Support vector machines                                          NaN   

                                                                               \
                                 Artificial neural networks Bayesian networks   
Artificial neural networks                              NaN               NaN   
Cluster analysis algorithms                             NaN               NaN   
Clustering criteria                                     NaN               NaN   
Datasets in computer vision                             NaN               NaN   
Decision trees                                          NaN               NaN   
Deep learning                                            {}               NaN   
Ensemble learning                                       NaN               NaN   
Factor analysis                                         NaN               NaN   
Gene expression programming                             NaN               NaN   
Genetic algorithms                                      NaN               NaN   
Genetic programming                                     NaN               NaN   
Graphical models                                        NaN 

In [64]:
pd.DataFrame.from_dict( test)

,machine learning
Applied machine learning,{}
Artificial intelligence conferences,{}
Artificial neural networks,"{'Deep learning': {}, 'Neural network software..."
Bayesian networks,{}
Classification algorithms,{'Artificial neural networks': {'Deep learning...
Cluster analysis,"{'Cluster analysis algorithms': {}, 'Clusterin..."
Computational learning theory,{}
Data mining and machine learning software,{'Social network analysis software': {}}
Datasets in machine learning,{'Datasets in computer vision': {}}
Dimension reduction,{'Factor analysis': {}}


In [8]:
visited = []

category_df = request_elements( 'machine learning', 'subcat')

subcats = [cat for cat in category_df.title.str.replace('Category:', '').tolist() if cat not in visited]

visited += subcats

output = {}


['Applied machine learning',
 'Artificial neural networks',
 'Bayesian networks',
 'Classification algorithms',
 'Cluster analysis',
 'Computational learning theory',
 'Artificial intelligence conferences',
 'Data mining and machine learning software',
 'Datasets in machine learning',
 'Dimension reduction',
 'Ensemble learning',
 'Evolutionary algorithms',
 'Genetic programming',
 'Inductive logic programming',
 'Kernel methods for machine learning',
 'Latent variable models',
 'Learning in computer vision',
 'Log-linear models',
 'Loss functions',
 'Machine learning algorithms',
 'Machine learning portal',
 'Machine learning task',
 'Markov models',
 'Machine learning researchers',
 'Semisupervised learning',
 'Statistical natural language processing',
 'Structured prediction',
 'Supervised learning',
 'Support vector machines',
 'Unsupervised learning']

In [7]:
get_cat_structure( 'machine learning')

AttributeError: 'DataFrame' object has no attribute 'title'

In [ ]:
visited = []

category_df = request_elements( 'machine learning', 'subcat')
    
subcats = [cat for cat in category_df.title.str.replace('Category:', '').tolist() if cat not in visited]

subcats

In [ ]:
get_subcategories1( 'machine learning', visited = [])

In [ ]:
subcategories, catdict = get_subcategories1( 'machine learning', 4, subcategories = [], catdict = {})

In [ ]:
catdict